In [1]:
import pdb
import pickle
import string

import time

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import scipy
import sklearn
from gensim.models import KeyedVectors
from nltk.corpus import stopwords, twitter_samples

from utils import (cosine_similarity, get_dict)
from os import getcwd

The code to download all required data was taken from Coursera 
https://github.com/amanchadha/coursera-natural-language-processing-specialization/blob/master/1%20-%20Natural%20Language%20Processing%20with%20Classification%20and%20Vector%20Spaces/Week%204/C1W4_A1_Word%20Translation.ipynb

In [8]:
!curl -o ./wiki.multi.fr.vec https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.fr.vec

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0  599M    0 34068    0     0  18805      0  9:17:14  0:00:01  9:17:13 18968
  0  599M    0  100k    0     0  36585      0  4:46:25  0:00:02  4:46:23 36788
  0  599M    0 2696k    0     0   714k      0  0:14:18  0:00:03  0:14:15  718k
  1  599M    1 7480k    0     0  1567k      0  0:06:31  0:00:04  0:06:27 1572k
  1  599M    1 11.7M    0     0  2083k      0  0:04:54  0:00:05  0:04:49 2714k
  2  599M    2 16.0M    0     0  2428k      0  0:04:12  0:00:06  0:04:06 3307k
  3  599M    3 20.0M    0     0  2646k      0  0:03:52  0:00:07  0:03:45 4120k
  4  599M    4 24.8M    0     0  2896k      0  0:03:31  0:00:08  0:03:23 4541k
  4  599M    4 29.4M    0     0  3085k      0  0:03

In [2]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [15]:
# Use this code to download and process the full dataset on your local computer

from gensim.models import KeyedVectors

en_embeddings = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary = True)
fr_embeddings = KeyedVectors.load_word2vec_format('./wiki.multi.fr.vec')

english_set = set(en_embeddings.index_to_key)
french_set = set(fr_embeddings.index_to_key)
en_embeddings_subset = {}
fr_embeddings_subset = {}
french_words = set(en_fr_train.values())

for en_word in en_fr_train.keys():
    fr_word = en_fr_train[en_word]
    if fr_word in french_set and en_word in english_set:
        en_embeddings_subset[en_word] = en_embeddings[en_word]
        fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]


for en_word in en_fr_test.keys():
    fr_word = en_fr_test[en_word]
    if fr_word in french_set and en_word in english_set:
        en_embeddings_subset[en_word] = en_embeddings[en_word]
        fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]


pickle.dump( en_embeddings_subset, open( "en_embeddings.p", "wb" ) )
pickle.dump( fr_embeddings_subset, open( "fr_embeddings.p", "wb" ) )


The length of the english to french training dictionary is 5000
The length of the english to french test dictionary is 5000


In [3]:
# loading the english to french dictionaries
en_fr_train = get_dict('en-fr.train.txt')
print('The length of the english to french training dictionary is', len(en_fr_train))
en_fr_test = get_dict('en-fr.test.txt')
print('The length of the english to french test dictionary is', len(en_fr_train))
en_embeddings_subset = pickle.load(open("en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("fr_embeddings.p", "rb"))

The length of the english to french training dictionary is 5000
The length of the english to french test dictionary is 5000


In [44]:
#combines all enlish and french vectors into respective matrices
def get_matrices(en_fr, french_vecs, english_vecs):
    # this provides 2 matrices:
    # X: columns are english embeddings
    # Y: columns are french embeddings

    X=[]
    Y=[]
    eng_terms = set(english_vecs.keys())
    french_terms = set(french_vecs.keys())

    for eng, fr in en_fr.items():
        if eng in eng_terms and fr in french_terms:
            X.append(english_vecs[eng])
            Y.append(french_vecs[fr])

    X = np.asarray(X)
    Y = np.asarray(Y)

    return X, Y


In [45]:
X_train, Y_train = get_matrices(en_fr_train,fr_embeddings_subset,en_embeddings_subset)

In [7]:
# frobenius loss function
def loss_fn(X,R,Y):
    A = X@R-Y
    sum = 0
    m=X.shape[0]
    for row in A:
        for cell in row:
            sum+=cell**2

    return sum/m

#gradient function
def gradient(X,R,Y):

    m = X.shape[0]
    return 2 * X.T @ (X@R-Y) / m

#provides matrix R, a transformation bw the english and french vector space
def align_embeddings(X,Y, train_steps = 100, learning_rate = 0.0003):
    np.random.seed(30)
    
    R = np.random.rand(X.shape[1],X.shape[1])
    
    for i in range(1,train_steps+1):
        if i%25==0:
            print(f'loss at {i}th iteration is: {loss_fn(X,R,Y)}')
        R-= learning_rate * gradient(X,R,Y)
    
    return R

    


In [43]:
Y_train.shape

(9864, 300)

In [49]:
R = align_embeddings(X_train,Y_train,train_steps=1000,learning_rate=0.8)

loss at 25th iteration is: 103.4231890420284
loss at 50th iteration is: 27.983561727823485
loss at 75th iteration is: 10.14778893904696
loss at 100th iteration is: 4.519926914719542
loss at 125th iteration is: 2.3942884774312243
loss at 150th iteration is: 1.4823453010613057
loss at 175th iteration is: 1.0529765592879439
loss at 200th iteration is: 0.8364359919468837
loss at 225th iteration is: 0.7214002414291245
loss at 250th iteration is: 0.6577456329502689
loss at 275th iteration is: 0.6213332103227337
loss at 300th iteration is: 0.5999129481174299
loss at 325th iteration is: 0.5870035746969205
loss at 350th iteration is: 0.5790565056333816
loss at 375th iteration is: 0.5740715249768429
loss at 400th iteration is: 0.5708921566297782
loss at 425th iteration is: 0.5688343935218162
loss at 450th iteration is: 0.5674852607961616
loss at 475th iteration is: 0.5665906961004559
loss at 500th iteration is: 0.5659916991397081
loss at 525th iteration is: 0.5655872004640026
loss at 550th itera

In [41]:
#nearest_neighbours

#naive knn
def knn(v,candidates,k=1): # returns indices of the k nearest neighbours
    similarities=[]
    for u in candidates:
        similarities.append(cosine_similarity(v,u))
    
    return np.argsort(similarities)[-k:]

In [37]:
#test accuracy 
import pandas as pd
def test_accuracy(X,R,Y):

    pred = X@R
    acc=0
    n = len(pred)
    for i in range(n):
        acc+=(i==knn(pred[i],Y)[0])
    
    return acc/n

In [47]:
X_test,Y_test = get_matrices(en_fr_test,fr_embeddings_subset,en_embeddings_subset)

In [48]:
test_accuracy(X_test,R,Y_test)

0.547287899860918

In [50]:
test_accuracy(X_train,R,Y_train)

0.5539334955393349